# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0  92995      0 --:--:-- --:--:-- --:--:-- 92839


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   182k      0 --:--:-- --:--:-- --:--:--  183k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [6]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [7]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [8]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [9]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [10]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 68)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 68)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [11]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


$\color{blue}{\textsf{[Answer]}}$

- `SingleHopSpecificQuerySynthesizer`: generates queries that involve a single, direct relationship or "hop" in the knowledge graph.  Looking at a single piece of information or a direct connection in the data.

- `MultiHopAbstractQuerySynthesizer`: creates queries that require traversing multiple relationships or "hops" in the knowledge graph, but in a more abstract manner.  Suitable for complex questions that need synthesizing information from multiple sources or steps, but the queries are more generalized and not tied to specific details.

- `MultiHopSpecificQuerySynthesizer`: Similar to the MultiHopAbstractQuerySynthesizer, this one also involves multiple hops, but it focuses on generating queries with specific details.  For detailed and complex questions that require piecing together specific information from various parts of the knowledge graph.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [12]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What happened to GPT-4's standing in the AI mo...,[Prompt driven app generation is a commodity a...,GPT-4-0314 has fallen to around 70th place on ...,single_hop_specifc_query_synthesizer
1,What details are provided about the butterflie...,"[gets you OpenAI’s most expensive model, o1. G...",The description of the butterflies at the Cali...,single_hop_specifc_query_synthesizer
2,Wut is Anthropic's Claude Artifacts and how do...,[your camera feed with the model and talk abou...,Anthropic's Claude Artifacts is a groundbreaki...,single_hop_specifc_query_synthesizer
3,What Anthropic doin' with evals and system pro...,[“agents” as dependent on AGI itself. A model ...,Anthropic's Amanda Askell emphasizes the impor...,single_hop_specifc_query_synthesizer
4,what microsoft doin with datacenters and envir...,[day after that. DeepSeek v3 is a huge 685B pa...,Companies like Microsoft are spending billions...,single_hop_specifc_query_synthesizer
5,How do inference-scaling reasoning models and ...,[<1-hop>\n\nPrompt driven app generation is a ...,"Inference-scaling reasoning models, such as Op...",multi_hop_abstract_query_synthesizer
6,How has the development of real-time camera fe...,[<1-hop>\n\nPrompt driven app generation is a ...,The development of real-time camera feed analy...,multi_hop_abstract_query_synthesizer
7,How do inference-scaling reasoning models and ...,[<1-hop>\n\nPrompt driven app generation is a ...,"Inference-scaling reasoning models, such as Op...",multi_hop_abstract_query_synthesizer
8,How did the advancements in LLMs in 2023 and 2...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2023 and 2024, advancements in Large Langua...",multi_hop_specific_query_synthesizer
9,How did the Claude 3 series contribute to the ...,[<1-hop>\n\nPrompt driven app generation is a ...,"The Claude 3 series, launched by Anthropic in ...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [14]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,what meta do with LLMs now?,[Voice and live camera mode are science fictio...,Meta is one of the 18 organizations with model...,single_hop_specifc_query_synthesizer
1,What are the recent advancements in OpenAI's m...,[the then-new GPT-4 Turbo and $1/mTok for GPT-...,OpenAI has introduced audio input and output c...,single_hop_specifc_query_synthesizer
2,What recent advancements have been made in Cha...,[live video. ChatGPT voice mode now provides t...,ChatGPT voice mode now provides the option to ...,single_hop_specifc_query_synthesizer
3,what anthropic do with evals and prompts?,[September 2022. I’m beginning to see the most...,Anthropic's Amanda Askell emphasizes the impor...,single_hop_specifc_query_synthesizer
4,How do the challenges of understanding and con...,[<1-hop>\n\nCode may be the best application T...,The challenges of understanding and controllin...,multi_hop_abstract_query_synthesizer
5,How do the challenges of understanding and con...,[<1-hop>\n\nCode may be the best application T...,The challenges of understanding and controllin...,multi_hop_abstract_query_synthesizer
6,How do the challenges of understanding and con...,[<1-hop>\n\nCode may be the best application T...,The challenges of understanding and controllin...,multi_hop_abstract_query_synthesizer
7,How have the training costs and environmental ...,[<1-hop>\n\nCode may be the best application T...,"The training costs for AI models, particularly...",multi_hop_abstract_query_synthesizer
8,How has the development of GPT-4o and its pric...,[<1-hop>\n\nthe then-new GPT-4 Turbo and $1/mT...,The development of GPT-4o has significantly im...,multi_hop_specific_query_synthesizer
9,What were the key developments in large langua...,[<1-hop>\n\neasy to follow. The rest of the do...,"In Simon Willison's reviews, several key devel...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [15]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [16]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [17]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [19]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [20]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [21]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [22]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [24]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [25]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents are described as a vague term in the context of AI that typically refers to "AI systems that can go away and act on your behalf." However, the term lacks a single, clear, and widely understood definition, leading to various interpretations. Some define agents as similar to a travel agent model, while others think of them in terms of large language models (LLMs) that have access to tools for solving problems. The term "autonomy" is also often associated with agents, but again, without a clear definition. Overall, there is skepticism about the utility of agents due to challenges related to gullibility and the inability of current AI systems to reliably distinguish truth from fiction.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [26]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [27]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: Evaluates the overall quality of the question-answer (QA) interaction. In other words, it judges whether the answer is correct or relevant in the context of the question.

- `labeled_helpfulness_evaluator`: Evaluates the helpfulness of the answer based on the provided reference answer. It checks if the answer is helpful to the user by considering the correctness of the reference answer.

- `dope_or_nope_evaluator`: Evaluates the coolness or coolness of the answer. It checks if the answer is cool or not.

## LangSmith Evaluation

In [28]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'roasted-middle-74' at:
https://smith.langchain.com/o/b2bbbf41-ce3b-46fa-8cc0-7de9bb768aa9/datasets/118e626a-7b65-4d94-9f1b-626a934dfabd/compare?selectedSessions=e359259a-f5a2-4a8f-9ca1-c28486b6b2e3




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How did the advancements in large language mod...,I don't know.,None,"In 2023 and 2024, significant advancements in ...",0,0,0,2.450803,ce97cfa0-b2ad-492b-9c4e-6960ff05a8d7,cfa96612-7913-470d-9b9f-29370ba349c3
1,How have the developments in GPT-4o and GPT-4V...,I don't know.,None,The developments in GPT-4o and GPT-4V models h...,0,0,0,2.660989,fa624589-0f82-4701-ac1b-0de1aa560220,af6b140f-c43c-4e98-b2fe-2dd0532ca337
2,What were the key developments in large langua...,"The key developments in large language models,...",None,"In Simon Willison's reviews, several key devel...",1,0,1,7.784699,c39b7fd9-f891-4bb5-a514-a1cf762e24c9,5d0359f8-3de9-485b-bf10-bb05ca3024b0
3,How has the development of GPT-4o and its pric...,The development of GPT-4o has increased effici...,None,The development of GPT-4o has significantly im...,1,0,0,5.318961,83dd5560-c06d-4d78-a518-9188aee4759d,6dc7e4ac-fe7c-43ac-a7e5-44a7b4ffe827
4,How have the training costs and environmental ...,The training costs for AI models have decrease...,None,"The training costs for AI models, particularly...",1,0,0,4.429235,6534f6a8-dea4-4601-bff4-a743e1e93329,6bd327cc-028b-4f74-ae34-a0cbe3fb332c
5,How do the challenges of understanding and con...,I don't know.,None,The challenges of understanding and controllin...,0,0,0,3.069575,77e78ecd-26c7-4539-b235-c6c75e5e98d7,c03968df-f2b9-4661-8bd9-3c6e65174c08
6,How do the challenges of understanding and con...,I don't know.,None,The challenges of understanding and controllin...,0,0,0,1.830175,3d6abdf4-b0d8-46d5-a916-dd4ebc51d2e1,213901b5-4aff-43ed-b542-01483a539510
7,How do the challenges of understanding and con...,The challenges of understanding and controllin...,None,The challenges of understanding and controllin...,1,0,0,4.093593,73b47950-1b03-4c50-8901-571e63cf8901,4fdf4813-2ec7-4993-8239-4a5643273c9e
8,what anthropic do with evals and prompts?,Anthropic emphasizes the importance of evaluat...,None,Anthropic's Amanda Askell emphasizes the impor...,1,1,0,4.502989,00cba128-4ecd-4a36-9e61-2dd78be87707,497a8202-d908-418a-b138-aebb89f9dcf2
9,What recent advancements have been made in Cha...,Recent advancements in ChatGPT's capabilities ...,None,ChatGPT voice mode now provides the option to ...,1,1,1,3.479423,6937ef85-9d85-4451-8ac0-dd8fd02bd375,65211aa3-75ee-48de-aa49-121c6a741cf1


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [29]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [30]:
rag_documents = docs

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?


$\color{blue}{\textsf{[Answer]}}$

Amount of Context:

A larger chunk size means that each piece of text sent to the language model contains more context. This can help the model generate more detailed and accurate answers because it has access to a larger portion of the original information.

Relevance and Noise:

However, larger chunks can also include extraneous details or noise alongside the relevant information. This may occasionally reduce the precision of the answer if too much irrelevant context is included. In contrast, a smaller chunk size might capture a more focused piece of information but could miss out on the broader context needed to answer a complex query.

Token Limits and Processing Efficiency:

Language models have token limits. If the chunks are too large, they might approach or exceed these limits, potentially leading to truncated inputs or slower processing times. Conversely, very small chunks may be more efficient to process but could lead to incomplete information being considered for answering questions.


In [32]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

$\color{blue}{\textsf{[Answer]}}$

- Modifying the embedding model modifies the way the text is represented as vectors.  A larger embedding model means that the text is represented in a more detailed way, which can lead to more accurate and detailed answers.


In [33]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [66]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [38]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [39]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

"Yo, so “agents” are this super ambiguous term in AI land. People toss it around to mean a bunch of different things, like AI systems that act on your behalf—think travel agents—or LLMs using tools to solve problems. But here’s the kicker: nobody seems to agree on what the term really means, which just makes things even more confusing! Basically, it's like trying to catch smoke with your bare hands—almost everyone’s excited about them, but examples in the wild are still kinda ghostly. So, yeah, agents are all about that potential but haven’t quite rolled out yet."

Finally, we can evaluate the new chain on the same test set!

In [40]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'ordinary-zipper-1' at:
https://smith.langchain.com/o/b2bbbf41-ce3b-46fa-8cc0-7de9bb768aa9/datasets/118e626a-7b65-4d94-9f1b-626a934dfabd/compare?selectedSessions=06e433db-2f61-4156-a1b6-ebcdaebff8b0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How did the advancements in large language mod...,I don't know.,None,"In 2023 and 2024, significant advancements in ...",0,0,```,1.633982,ce97cfa0-b2ad-492b-9c4e-6960ff05a8d7,8d299df3-6907-4dfa-b1d5-08713d0e3620
1,How have the developments in GPT-4o and GPT-4V...,"Yo, here's the scoop: GPT-4o has those sweet a...",None,The developments in GPT-4o and GPT-4V models h...,1,0,1,3.479578,fa624589-0f82-4701-ac1b-0de1aa560220,65bf7bf0-3c81-4526-9410-150e9aee490c
2,What were the key developments in large langua...,"Yo! So, in Simon Willison's epic reviews of 20...",None,"In Simon Willison's reviews, several key devel...",0,0,1,4.327197,c39b7fd9-f891-4bb5-a514-a1cf762e24c9,978ed4a4-7bf5-43c1-b111-4a69229eb369
3,How has the development of GPT-4o and its pric...,"Yo, check it out! The evolution of GPT-4o has ...",None,The development of GPT-4o has significantly im...,1,0,1,4.913471,83dd5560-c06d-4d78-a518-9188aee4759d,a0800aff-b3e8-44d2-8f9c-f3ecdcb55c2a
4,How have the training costs and environmental ...,"Yo! So, check it out—training costs for AI mod...",None,"The training costs for AI models, particularly...",1,1,1,3.834665,6534f6a8-dea4-4601-bff4-a743e1e93329,f57379ff-a225-4939-8f26-3c1bcad66b5d
5,How do the challenges of understanding and con...,"Yo, the challenges of understanding and contro...",None,The challenges of understanding and controllin...,1,0,1,3.888242,77e78ecd-26c7-4539-b235-c6c75e5e98d7,0996f4c5-405d-4b0f-a40a-e1d681f8eb11
6,How do the challenges of understanding and con...,The challenges of understanding and controllin...,None,The challenges of understanding and controllin...,1,0,1,4.379034,3d6abdf4-b0d8-46d5-a916-dd4ebc51d2e1,2a7da10b-27ef-4fe8-a641-728c81c317b7
7,How do the challenges of understanding and con...,"Yo, the challenges of wrapping our heads aroun...",None,The challenges of understanding and controllin...,1,0,1,3.476395,73b47950-1b03-4c50-8901-571e63cf8901,696dd1dc-469f-4770-bb86-9f5149d4ad3b
8,what anthropic do with evals and prompts?,Anthropic emphasizes the importance of evals i...,None,Anthropic's Amanda Askell emphasizes the impor...,1,1,1,5.119831,00cba128-4ecd-4a36-9e61-2dd78be87707,8d1feffa-ebd8-49cf-bab9-afe5ec0eba1f
9,What recent advancements have been made in Cha...,"Yo, check it out! Recently, ChatGPT leveled up...",None,ChatGPT voice mode now provides the option to ...,1,1,1,8.394059,6937ef85-9d85-4451-8ac0-dd8fd02bd375,3782bba4-7051-4eb4-a574-a3db03599d00


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

   ![LangSmit Evaluation Comparison](LangSmit-eval-comparison.png)

Correctness:

The increase in correctness could be due to better retrieval of relevant context, thanks to the improved embedding model and larger chunk sizes. These changes allow the model to access more detailed and accurate information, leading to more correct answers.

Dopeness:

The increase in the "dopeness" metric likely results from the "dope" prompt augmentation. This prompt encourages the model to generate responses that are not only correct but also engaging and stylistically appealing, which can enhance the perceived quality of the answers.

Helpfulness:

The improvement in helpfulness might be attributed to the combination of better context retrieval and the focus on generating answers that are both correct and engaging. The enhanced embeddings and chunk sizes ensure that the model has access to comprehensive information, making the answers more useful to users.

Overall, these changes suggest that the modifications to the RAG chain—such as using a larger embedding model, increasing chunk sizes, and adding stylistic prompts—have positively impacted the model's ability to generate accurate, engaging, and helpful responses.